In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# check API keys
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
# Initialize OpenAI client
openai = OpenAI()

In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are a snarky assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
# See how this function creates exactly the format above

def messages_for(website):
     return [
        {"role": "system", "content": system_prompt},
         {"role": "user", "content": user_prompt_for(website)}
     ]

In [8]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [9]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [13]:
display_summary("https://www.jair.org")

# Summary of Journal of Artificial Intelligence Research

The **Journal of Artificial Intelligence Research (JAIR)** is an open-access platform founded in 1993, aimed at the rapid dissemination of significant research in all areas of artificial intelligence (AI). The journal covers diverse topics including agents, automated reasoning, machine learning, natural language processing, robotics, and uncertainty in AI.

## Current Issue Highlights
- **Vol. 83 (2025)** features various articles such as:
  - **Geometrically Inspired Kernel Machines for Collaborative Learning**
  - **The Search for Stability: Learning Dynamics of Strategic Publishers**
  - **LabelCoRank: Long Tail Multi-Label Classification**
  - **Towards Generalist Robot Learning from Internet Video: A Survey**
  - **Improved Regret Bounds for Bandits with Expert Advice**

## News and Announcements
- **AAAI Contributes to JAIR Sustainability Campaign** - A collaborative initiative to ensure the journal's sustainability.
- **Reproducibility Initiative** - A program encouraging the reproducibility of AI research.
- **2024 IJCAI-JAIR Prize Awarded** - Recognizes outstanding contributions.
- **Special Track on Multi-Agent Path Finding** - A focused exploration of multi-agent systems.
- **JAIR Available in ACM Library** - Increasing accessibility of JAIR publications.

For submissions, JAIR invites papers that uphold high standards of originality and significance in AI research. The journal continues to promote the dissemination of scientific results while being a leading publication in the field.

In [14]:
display_summary("https://arxiv.org/list/cs.AI/recent")

# Summary of Artificial Intelligence Website

The **Artificial Intelligence** website serves as a repository for various research articles and papers focused on the field of artificial intelligence. It primarily features **recent submissions** from authors, providing a total of **814 entries** that cover a wide range of topics, including safe AI systems, performance of concept probing, reinforcement learning applications, and logical characterizations of neural networks, among many others.

## Recent Submissions Highlights
1. **SafeWork-R1: Coevolving Safety and Intelligence under the AI-45° Law**:
   - Authors: Shanghai AI Lab 
   - Focus: Safety measures in AI development.
   - [Link to Paper](arXiv:2507.18576)

2. **On the Performance of Concept Probing: The Influence of the Data (Extended Version)**:
   - Authors: Manuel de Sousa Ribeiro, Afonso Leote, João Leite
   - Focus: Analysis of performance metrics based on data.
   - [Link to Paper](arXiv:2507.18550)

3. **Optimising Call Centre Operations using Reinforcement Learning: Value Iteration versus Proximal Policy Optimisation**:
   - Authors: Kwong Ho Li, Wathsala Karunarathne
   - Focus: Comparison of reinforcement learning techniques for operational efficiency.
   - [Link to Paper](arXiv:2507.18398)

4. **Foundations for Risk Assessment of AI in Protecting Fundamental Rights**:
   - Authors: Antonino Rotolo, Beatrice Ferrigno, etc.
   - Focus: Exploring the intersection of AI and fundamental human rights.
   - [Link to Paper](arXiv:2507.18290)

5. **SMARTAPS: Tool-augmented LLMs for Operations Management**:
   - Authors: Timothy Tin Long Yu et al.
   - Focus: Innovations in operations management using LLMs (Large Language Models).
   - [Link to Paper](arXiv:2507.17927)

### Noteworthy Updates
- Several papers have been accepted for upcoming conferences, including **ACM MM 2025** and **IEEE International Symposium on Personal, Indoor and Mobile Radio Communications 2025**.

Overall, the website acts as a crucial hub for researchers interested in the latest advancements and discussions in AI, providing both foundational theories and practical applications.